In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import os
import time
from scipy.sparse import csr_matrix
import pandas as pd

In [2]:
output_dir = 'lp_files_gap'
os.makedirs(output_dir, exist_ok=True)

res_dir = 'results'
os.makedirs(res_dir, exist_ok=True)

# Basis Matrix

In [3]:
def construct_b_matrix_with_slack(model, tolerance=1e-10):
    """
    Constructs the basis matrix (B matrix) for a linear programming problem from the Gurobi model. 
    The B matrix includes columns corresponding to basic variables and slack variables for the constraints.

    This matrix is used in simplex-based algorithms to represent the current basic feasible solution. 
    The function also computes and returns the inverse of the B matrix.

    Args:
        model (gurobipy.Model): The solved Gurobi model.
        tolerance (float): The tolerance level to filter small values in the inverse of the B matrix (B_inv). 
                           Values smaller than this tolerance will be set to 0.

    Returns:
        B_inv (np.ndarray): The inverse of the B matrix after including slack columns. If the matrix is 
                            singular, None is returned instead.
        B_matrix (np.ndarray): The constructed B matrix including slack columns.

    Behavior:
    1. Retrieves the basis status of the variables from the Gurobi model, identifying basic variables.
    2. Builds the B matrix by extracting the coefficients of basic variables from the model's constraint matrix (A matrix).
    3. Slack columns are added to account for non-zero slack values in the model's constraints.
    4. The function attempts to invert the B matrix. If the inversion is successful, small values in the inverse 
       are filtered out using the given tolerance.
    5. Returns both the B matrix and its inverse. If the inversion fails (e.g., the matrix is singular), `None` 
       is returned for B_inv.

    Notes:
    - The function assumes that the model has already been solved and the basis status is available.
    - The A matrix (constraint coefficients) is assumed to be accessible via `model.getA()`, which returns the constraint matrix.
    """

    # Get the basis status of the variables
    basis_status = model.getAttr('VBasis')

    # Identify the basic variables (basis status == 0)
    basic_vars = [i for i, status in enumerate(basis_status) if status == 0]

    # Get the slack values for each constraint
    slacks = [constr.Slack for constr in model.getConstrs()]

    # Assuming A matrix is in sparse format and obtained from the model
    A = model.getA()

    # Step 1: Add rows from A corresponding to basic variables
    B_matrix = []
    for constr_index in range(A.shape[0]):  # Iterate over constraints
        row = []
        for var_index in range(A.shape[1]):  # Iterate over all variables
            if var_index in basic_vars:
                # Add the coefficient for basic variables
                coef = A[constr_index, var_index] if (constr_index < A.shape[0] and var_index < A.shape[1]) else 0
                row.append(coef)
        B_matrix.append(row)

    # Step 2: Count non-zero slacks and add slack columns
    non_zero_slack_count = sum(1 for slack in slacks if slack > 0)
    B_matrix = np.array(B_matrix)
    
    # Initialize an identity matrix for slack columns
    slack_columns = np.zeros((B_matrix.shape[0], non_zero_slack_count))

    # Fill the slack columns for each non-zero slack
    slack_col_index = 0
    for constr_index, slack_value in enumerate(slacks):
        if slack_value > 0:  # Check for non-zero slack
            slack_columns[constr_index, slack_col_index] = 1  # Set 1 for the corresponding slack variable row
            slack_col_index += 1

    # Concatenate the slack columns to the B matrix
    B_matrix = np.hstack((B_matrix, slack_columns))

    # Invert the B matrix
    try:
        B_inv = np.linalg.inv(B_matrix)
    except np.linalg.LinAlgError:
        return None, B_matrix

    # Apply tolerance to small values in the inverse
    B_inv[np.abs(B_inv) < tolerance] = 0  

    return B_inv, B_matrix


# Gomory's cuts

In [4]:
import numpy as np
import gurobipy as gp

def is_fractional(value, tolerance=1e-5):
    """
    Checks if a value is fractional within a given tolerance.

    Args:
        value (float): The value to check.
        tolerance (float): The tolerance for checking if the value is considered fractional.

    Returns:
        bool: True if the value is fractional (i.e., not an integer), False otherwise.
    """
    fractional_part = np.abs(value - np.round(value))
    return fractional_part > tolerance

def add_gomory_cuts(model, B_inv, curr_obj, tolerance=1e-5):
    """
    Adds Gomory fractional cuts to a Gurobi model to improve the solution's integrality. 
    If no improvement is made in the objective after adding a cut, it removes the cut.

    Args:
        model (gurobipy.Model): The solved Gurobi model from which to add Gomory cuts.
        B_inv (np.ndarray): The inverse of the basis matrix (B matrix).
        curr_obj (float): The current objective value before adding Gomory cuts.
        tolerance (float): The tolerance level for filtering small values in B_inv and checking for improvement.

    Returns:
        (model, B_inv): Updated Gurobi model and the inverse of the basis matrix if cuts are added successfully. 
                        Returns None for B_inv if no improvements are made or if the model becomes infeasible/unbounded.

    Behavior:
    1. Iterates over each row of the inverse B matrix.
    2. Checks for fractional values within the row using the `is_fractional` function.
    3. Constructs Gomory cuts for constraints associated with fractional values found.
    4. Adds Gomory cuts to the model as new constraints and optimizes the model.
    5. If the objective value does not improve significantly, removes the Gomory cut.
    6. Returns the updated model and B_inv, or None if no improvements are made or the model becomes infeasible/unbounded.

    Notes:
    - The function assumes the Gurobi model has already been solved and is ready for cut addition.
    - The function updates the model after each addition or removal of a Gomory cut.
    """
    
    num_rows = model.NumConstrs  # Get the number of constraints

    # Get the A matrix and RHS values from the model
    A_matrix = model.getA()
    rhs_values = model.getAttr('RHS', model.getConstrs())

    # Get all variables from the model
    variables = model.getVars()

    # Iterate over each row of B^-1
    for i in range(num_rows):
        binv_row = B_inv[i]  # Get the i-th row of B^-1

        # Check for fractional values in the row
        if np.any([is_fractional(val, tolerance) for val in binv_row]):
            # print(f"Fractional part exists in row {i}.")
            gomory_cut_expr = gp.LinExpr()
            gomory_cut_rhs = 0  # Initialize the RHS

            var_coeffs = {}

            # Iterate through each value in the binv_row and corresponding constraints
            for j in range(len(binv_row)):
                if j < A_matrix.shape[0]:  # Ensure we are within the constraints
                    coeffs = A_matrix.getrow(j).data  # Coefficients of the j-th constraint
                    vars_in_row = A_matrix.getrow(j).indices  # Variable indices of the j-th constraint

                    binv_row[j] = round(binv_row[j], 5)
                    fractional_part = binv_row[j] - np.floor(binv_row[j])
                    
                    if fractional_part > 1e-5:  # Ignore very small fractions
                        for k in range(len(coeffs)):
                            var_index = vars_in_row[k]  # Variable index
                            coefficient = fractional_part * coeffs[k]
                            if var_index in var_coeffs:
                                var_coeffs[var_index] += coefficient
                            else:
                                var_coeffs[var_index] = coefficient

                        # Include the RHS value for the current constraint
                        rhs_value = rhs_values[j]
                        gomory_cut_rhs += fractional_part * rhs_value

            # Create a new expression for the cut
            floored_expr = gp.LinExpr()
            for var_index, coeff in var_coeffs.items():
                # floored_expr.addTerms(coeff, variables[var_index])
                floored_expr.addTerms(np.floor(coeff), variables[var_index])

            # Floor the RHS value
            floored_rhs = np.floor(gomory_cut_rhs)

            # Add the new Gomory cut as a constraint
            gomory_cut = model.addConstr(floored_expr <= floored_rhs, name=f"gomory_cut_row_{i}")
            # print(f"Added Gomory cut from row {i} with floored RHS = {floored_rhs}.")

            # Optimize the model after adding the cut
            model.optimize()
            
            # Check the optimization status to ensure the solution is feasible
            status = model.Status
            if status == gp.GRB.Status.OPTIMAL:
                new_obj = model.ObjVal
                # print(f"Objective improved to {new_obj} after adding Gomory cut.")
                
                # Check if there was significant improvement
                if np.abs(new_obj - curr_obj) <= tolerance:
                    # No improvement, remove the Gomory cut
                    # print(f"No improvement from Gomory cut, removing row {i}.")
                    model.remove(gomory_cut)
                    model.update()
                else:
                    b_inv, _ = construct_b_matrix_with_slack(model)
                    if b_inv is None:
                        # print("B^-1 matrix is singular, removing Gomory cut.")
                        model.remove(gomory_cut)
                        model.update()
                    else:
                        return model, b_inv
            elif status in [gp.GRB.Status.INFEASIBLE, gp.GRB.Status.UNBOUNDED]:
                # If the model becomes infeasible or unbounded, remove the cut
                # print(f"Model became infeasible/unbounded after adding Gomory cut, removing row {i}.")
                model.remove(gomory_cut)
                model.update()
                return model, None

    return model, None


In [5]:
def process_lp_files(lp_files_dir="lp_files_gap", time_limit=120, tolerance=1e-10):
    """
    Processes all LP files in the specified directory, optimizing each model
    until an integer solution is found or a time limit is reached.

    Args:
        lp_files_dir (str): Directory containing LP files.
        time_limit (int): Time limit for optimization in seconds.
        tolerance (float): Tolerance for checking integer solutions.

    Returns:
        None
    """
    # Traverse the 'lp_files_gap' folder
    # Initialize an empty list to store results
    results = []    
    for root, dirs, files in os.walk(lp_files_dir):
        files = sorted(files)
        for file in files:
            if file.endswith(".lp"):  # Ensure we're working with .lp files
                # Full path of the input LP file
                input_file_path = os.path.join(root, file)
                model = gp.read(input_file_path)
                model.setParam('OutputFlag', 0)

                # Set time limit for the optimization process
                model.setParam('TimeLimit', time_limit)

                # Print or process the file name (for debugging or logging)
                print(f"Processing file: {file}")

                # Optimize the model initially to capture the starting objective value
                previous_objective = None
                loop_start_time = time.time()
                model.optimize()
                initial_objective = model.ObjVal
                
                # Construct the B matrix and its inverse
                b_inv, B_matrix = construct_b_matrix_with_slack(model, tolerance=tolerance)

                # Loop to solve the model until an integer solution is found or time limit is reached
                while True:
                    # Solve the model
                    model.optimize()

                    # Check if the solution is integer
                    is_integer = all(abs(v.x - round(v.x)) < 1e-10 for v in model.getVars())

                    # Check if the time limit has been reached
                    time_elapsed = time.time() - loop_start_time

                    # Break the loop if an integer solution is found or if the time limit has been reached
                    if previous_objective is not None:
                        if (model.ObjVal - previous_objective) < 1e-5:
                            break
                    if is_integer or time_elapsed >= time_limit:
                        break

                    # Store the current objective value for the next iteration
                    current_objective = model.ObjVal
                    previous_objective = current_objective

                    # Construct the B matrix and its inverse
                    b_inv, B_matrix = construct_b_matrix_with_slack(model, tolerance=tolerance)

                    # Add Gomory cuts to the model
                    if b_inv is not None:
                        model, b_inv = add_gomory_cuts(model, b_inv, current_objective, tolerance=1e-5)

                # Capture the final objective value after the loop
                final_objective = model.ObjVal
                
                # Store the results for the current file
                results.append({
                'file': file,
                'initial_value': initial_objective,
                'final_value': final_objective,
                'time_taken': time_elapsed
                })
                # print(f"Initial objective value: {initial_objective}")
                # print(f"Final objective value: {final_objective}")
                # print(f"Time elapsed: {time_elapsed:.2f} seconds")
                
                # # Print whether the solution is integer
                # if is_integer:
                #     print(f"Optimal integer solution found for file: {file}")
                # else:
                #     print(f"No optimal integer solution found for file: {file}")
                # print("")
    
    # Save the results to a CSV file   
    df = pd.DataFrame(results)
    df.to_csv(os.path.join(res_dir, 'gomory_cuts.csv'), index=False)
    print(f"Results saved to {res_dir}/gomory_cuts.csv")



In [6]:
if __name__ == "__main__":
    process_lp_files()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2544493
Academic license 2544493 - for non-commercial use only - registered to va___@iisc.ac.in
Read LP format model from file lp_files_gap/a05100.lp
Reading time = 0.03 seconds
obj: 105 rows, 500 columns, 1000 nonzeros
Processing file: a05100.lp
Initial objective value: 1697.7272727272727
Final objective value: 1698.0
Time elapsed: 2.04 seconds
Optimal integer solution found for file: a05100.lp

Read LP format model from file lp_files_gap/a05200.lp
Reading time = 0.02 seconds
obj: 205 rows, 1000 columns, 2000 nonzeros
Processing file: a05200.lp
Initial objective value: 3234.7391304347825
Final objective value: 3235.0
Time elapsed: 2.27 seconds
Optimal integer solution found for file: a05200.lp

Read LP format model from file lp_files_gap/a10100.lp
Reading time = 0.02 seconds
obj: 110 rows, 1000 columns, 2000 nonzeros
Processing file: a10100.lp
Initial objective value: 1358.556923076923
Final objective v